In [1]:
import warnings

import re
import pandas as pd
import numpy as np
import seaborn as sns
import nltk

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

warnings.filterwarnings("ignore")

In [2]:
#ЧЕКПОИНТ РЕШЕНИЯ
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submit = pd.read_csv("submission.csv")

nltk.download('stopwords')

#ПРЕДОБРАБОТКА ТЕКСТА
train = train.rename(columns={'Решение             ': "Решение"})
test = test.rename(columns={'Решение             ': "Решение"})

patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    return tokens

text_content = train['Содержание'].astype(str).str.lower().apply(lemmatize)
text_solution = train['Решение'].astype(str).str.lower().apply(lemmatize)
for i in range(text_content.shape[0]):
    text_content[i] = " ".join(text_content[i])
    text_solution[i] = " ".join(text_solution[i])
    
train.loc[:, "Содержание"] = text_content
train.loc[:, "Решение"] = text_solution
    
text_content = test['Содержание'].astype(str).str.lower().apply(lemmatize)
text_solution = test['Решение'].astype(str).str.lower().apply(lemmatize)
for i in range(text_content.shape[0]):
    text_content[i] = " ".join(text_content[i])
    text_solution[i] = " ".join(text_solution[i])

test.loc[:, "Содержание"] = text_content
test.loc[:, "Решение"] = text_solution

#НЕНУЖНЫЕ В БАН
train = train.drop(["Дата закрытия обращения", "Место"], axis=1)
test = test.drop(["Дата закрытия обращения", "Место"], axis=1)

#НОВЫЕ ГЕНЕРИРУЕМ
train.loc[:, "Дедлайн пропущен"] = pd.to_datetime(train["Дата восстановления"]) - pd.to_datetime(train["Крайний срок"])
train.loc[:, "Дедлайн пропущен"] = train["Дедлайн пропущен"].dt.days > 0
test.loc[:, "Дедлайн пропущен"] = pd.to_datetime(test["Дата восстановления"]) - pd.to_datetime(test["Крайний срок"])
test.loc[:, "Дедлайн пропущен"] = test["Дедлайн пропущен"].dt.days > 0

train.loc[:, "Время выполнения"] = pd.to_datetime(train["Дата восстановления"]) - pd.to_datetime(train["Дата обращения"])
test.loc[:, "Время выполнения"] = pd.to_datetime(test["Дата восстановления"]) - pd.to_datetime(test["Дата обращения"])

def date_std(array):
    return np.sqrt(np.sum((array - np.mean(array)) ** 2) / len(array))

data = train.groupby(by=["Критичность", "Влияние"])["Время выполнения"].aggregate(np.mean)
values_dict = {}

for i in range(len(data)):
    values_dict[data.index[i]] = data[i]

new_col = []
for row in range(len(train)):
    new_col.append(train.iloc[row, -1] - values_dict[(train.iloc[row, 12], train.iloc[row, 13])])
    
train.loc[:, "Кр./влияние"] = new_col
train.loc[:, "Кр./влияние"] = train['Кр./влияние'].astype("timedelta64[s]")

datа = test.groupby(by=["Критичность", "Влияние"])["Время выполнения"].aggregate(np.mean)
values_dict = {}

for i in range(len(data)):
    values_dict[data.index[i]] = data[i]

new_col = []
for row in range(len(test)):
    new_col.append(test.iloc[row, -1] - values_dict[(test.iloc[row, -5], test.iloc[row, -4])])
    
test.loc[:, "Кр./влияние"] = new_col
test.loc[:, "Кр./влияние"] = test['Кр./влияние'].astype("timedelta64[s]")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxxx\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [72]:
def remove_extra(elem):
    if elem not in '#$%&'+'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"' and len(elem) != 1:
        return True

def checker(x):
    result = []
    for content in x:
        result.append("тест" not in content)
    return result

def anomality_dataset_generation(data):
    #невозможные сроки выполнения
    total_dataset = pd.DataFrame()
    total_dataset = pd.concat([total_dataset, data[data["Время выполнения"].dt.total_seconds() < 0]], axis=0)
    
    #неправильная классификация (был запросом, на самом деле инцидент), либо сразу был инцидентом НАДО ЕЩЁ ДЛЯ ТЕСТА ТО ЖЕ САМОЕ
    data_only12 = data[(data["Тип переклассификации"] > 0)] #запрос->инцидент=1
    data_0_inc = data[(data["Тип переклассификации"] == 0) & (data["Тип обращения итоговый"] == "Инцидент")]

    #сборник плохих значений признака (аномалия)
    total_result = {}
    #колонки, по которым проверяемся
    main_cols = ["Содержание", "Сервис", "Статус", "Функциональная группа", "Решение", "Система"]

    #получаем словарь, где ключ - наименование колонки, значение - лист тех значений, которые позволяют записи быть аномальной
    for col in main_cols:
        subset = pd.DataFrame(data_only12.groupby(by=col))
        result={}
        col_index = list(data.columns).index(col)
        for i in range(len(subset)):
            cur_data = subset.iloc[i, 1]
            result[cur_data.iloc[0, col_index]] = (len(cur_data[cur_data["Тип переклассификации"] == 1]), len(cur_data[cur_data["Тип переклассификации"] == 2]))
        total_result[col] = [name for name in result.keys() if result[name][0] > 0 and result[name][1] == 0]
    
    #маска для проверки принадлежности записи к аномальной по значению колонки
        mask = []
        temp_data = data_only12.copy()
        for i in range(len(data_only12)):
            mask.append(data_only12.iloc[i, col_index] in total_result[col])
        temp_data.loc[:, "mask"] = mask
        temp_data = temp_data[temp_data["mask"] == True]#.drop("mask", axis=1)
    
    #подсоединение в общий датасет
        if len(total_dataset) == 0:
            total_dataset = temp_data.copy()
        else:
            total_dataset = pd.concat([total_dataset, temp_data], axis=0).reset_index().iloc[:, 1:]
    total_dataset = total_dataset.drop('mask', axis=1)
    total_dataset = pd.concat([total_dataset, data_0_inc], axis=0).reset_index().iloc[:, 1:]


    #аномальные запросы по системе/функциональной группе
    main_cols = ["Система", "Функциональная группа"]
    for col in main_cols:
        cur_data = pd.DataFrame(data.groupby(by=col))
        for row in range(len(cur_data)):
            subset = cur_data.iloc[row, 1]
            mask = subset["Содержание"].value_counts().reset_index().iloc[:, 1] < np.percentile(subset["Содержание"].value_counts().reset_index().iloc[:, 1].unique(), q=5)
            subset.loc[:, 'mask'] = mask
            subset = subset[subset['mask'] == True].drop('mask', axis=1)
            total_dataset = pd.concat([total_dataset, subset], axis=0)
    
    #причинно-следственные связи (проблема в банке / проблемы извне)
    df_subset = data[["Дата обращения", "Дата восстановления", "Содержание"]]
    df_subset = df_subset[checker(df_subset["Содержание"])]
    df_subset.loc[:, "Дата восстановления"] = pd.to_datetime(df_subset.loc[:, "Дата восстановления"])
    df_subset.loc[:, "Дата обращения"] = pd.to_datetime(df_subset.loc[:, "Дата обращения"])

    counts = []
    value_counts = df_subset["Содержание"].value_counts()
    for row in range(len(df_subset)):
        counts.append(value_counts[df_subset.iloc[row, 2]])
    df_subset.loc[:, "Количество повторений"] = counts
    df_subset = df_subset[df_subset["Содержание"] != "заявка №"]
    df_only1 = df_subset[df_subset["Количество повторений"] == 1].drop("Количество повторений", axis=1)
    df_subset = df_subset[df_subset["Количество повторений"] != 1]

    connect_dict = {}

    for content in df_subset["Содержание"].unique():
        content_set = set(content.split(' '))
        content_set = set(filter(remove_extra, content_set))
        for content1 in df_only1["Содержание"]:
            content1_set = set(content1.split(' '))
            content1_set = set(filter(remove_extra, content1_set))
            if len(content_set & content1_set) >= 2:
                if (content not in connect_dict.keys()):
                    connect_dict[content] = [content1]
                elif (content1 not in connect_dict[content]):
                    connect_dict[content].append(content1)
            else:
                if content not in connect_dict.keys():
                    connect_dict[content] = []

    summary_date = pd.DataFrame(data.groupby(by="Дата обращения")["Содержание"].value_counts())
    summary_date = summary_date.rename(columns={'Содержание': "Количество повторов в одно время"})
    summary_date = summary_date.reset_index()
    summary_date_only1 = summary_date[summary_date.iloc[:, -1] == 1]
    summary_date = summary_date[summary_date.iloc[:, -1] != 1]

    summary_date = summary_date[summary_date["Содержание"] != "заявка №"]
    summary_date_only1 = summary_date_only1.drop("Количество повторов в одно время", axis=1)

    summary_date = summary_date[checker(summary_date["Содержание"])]
    summary_date_only1 = summary_date_only1[checker(summary_date_only1["Содержание"])]

    summary_date = summary_date.reset_index().iloc[:, 1:]
    summary_date_only1 = summary_date_only1.reset_index().iloc[:, 1:]

    for i in range(len(summary_date)):
        if connect_dict[summary_date.iloc[i, 1]]:
            for content in connect_dict[summary_date.iloc[i, 1]]:
                subset = data[train["Содержание"] == content]
                subset = subset[(subset["Дата обращения"] < summary_date.iloc[i, 0]) & (subset["Дата восстановления"] > summary_date.iloc[i, 0])]
                total_dataset = pd.concat([total_dataset, subset], axis=0)
        else:
            total_dataset = pd.concat([total_dataset, data[(data["Содержание"] == summary_date.iloc[i,
                                                                        1]) & (data["Дата обращения"] == summary_date.iloc[i, 0])]], axis=0)
            
    
    dup = []
    total_dataset_wd = total_dataset.drop_duplicates()
    for ind in total_dataset_wd.index:
        dup.append(list(total_dataset.index).count(ind))
        
    total_dataset_wd.loc[:, "Частота"] = dup
    total_dataset_wd.loc[:, "Частота"] = total_dataset_wd.loc[:, "Частота"] / 5
    
    return total_dataset_wd

In [73]:
anomality_dataset_generation(test)

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,Решение,Тип переклассификации,Дата восстановления,Критичность,Влияние,Система,Дедлайн пропущен,Время выполнения,Кр./влияние,Частота
0,14833,заявка регистрироваться автоматический режим,Сервис11,3-Низкий,Закрыт,ФГ42,2018-02-22 06:47,2018-02-16 13:24,Запрос,Инцидент,заявка создать инцидент № список сметодержать ...,1,2018-02-17 08:34,4-Нет,3-Малое,Система11,False,0 days 19:10:00,-99404.0,0.2
1,15073,зависнуть запрос личный кабинет система,Сервис11,3-Низкий,Закрыт,ФГ42,2018-02-22 06:57,2018-02-17 03:58,Запрос,Инцидент,статус скорректировать причина разовый сбой си...,1,2018-02-17 08:43,4-Нет,3-Малое,Система11,False,0 days 04:45:00,-151304.0,0.4
2,15273,срочно заявка согласовать запрос система появи...,Сервис27,3-Низкий,Закрыт,ФГ42,2018-02-22 09:06,2018-02-17 07:51,Запрос,Инцидент,сбой система формирование выполнить перезапуск...,1,2018-02-17 09:26,3-Базовая,3-Малое,Система11,False,0 days 01:35:00,-133169.0,0.2
3,15287,согласование запрос создать система,Сервис11,3-Низкий,Закрыт,ФГ42,2018-02-23 08:36,2018-02-17 08:05,Запрос,Инцидент,заявка запрос инцидент №,1,2018-02-18 14:42,4-Нет,4-Нет влияния,Система11,False,1 days 06:37:00,-110631.0,0.2
4,15469,обращение произойти сбой регистрация,Сервис11,3-Низкий,Закрыт,ФГ42,2018-02-23 08:36,2018-02-17 10:48,Запрос,Инцидент,заявка создать инцидент №,1,2018-02-18 14:59,4-Нет,3-Малое,Система11,False,1 days 04:11:00,-66944.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6068,20871,предоставление техника брм,АРМ,3-Низкий,Закрыт,ФГ12,2018-03-14 06:51,2018-02-25 11:28,Запрос,Запрос,работа обращение выполнить,0,2018-03-07 11:31,4-Нет,4-Нет влияния,База,False,10 days 00:03:00,643329.0,0.2
6325,21128,предоставление техника брм,АРМ,3-Низкий,Закрыт,ФГ12,2018-03-15 12:36,2018-02-25 16:32,Запрос,Запрос,работа обращение выполнить,0,2018-03-09 11:16,4-Нет,4-Нет влияния,База,False,11 days 18:44:00,796989.0,0.2
6326,21129,предоставление техника брм,АРМ,3-Низкий,Закрыт,ФГ12,2018-03-22 06:36,2018-02-25 16:32,Запрос,Запрос,работа обращение выполнить,0,2018-03-16 07:38,4-Нет,4-Нет влияния,База,False,18 days 15:06:00,1388709.0,0.2
6327,21130,предоставление техника брм,АРМ,3-Низкий,Закрыт,ФГ12,2018-03-15 06:36,2018-02-25 16:32,Запрос,Запрос,работа обращение выполнить,0,2018-03-08 11:11,4-Нет,4-Нет влияния,База,False,10 days 18:39:00,710289.0,0.2
